In [2]:
import os
import re
import upload_v2 as v2

In [3]:
dict_refer = v2.dict_refer
comp = v2.comp

In [4]:
files = ["20429252 (주)주원 대출서류.pdf"]

for f in files:
    print("-----------------------------------------")
    f_name_items = {"key" :"", "name" :"", "docu" :"", "event" :"", "extra" :"", "date" :"", "pw" :""}
    extra = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    f_name_items["key"], extra = v2.keyFnc(extra) ############

    f_name_items["pw"], extra = v2.pwFnc(extra)  ###### pw 할당

    debtorName = dict_refer[f_name_items["key"]][2] 
    grtName = dict_refer[f_name_items["key"]][3]
    f_name_items["name"], extra = v2.nameFnc(extra, debtorName, grtName) ############
    print(f'name = {f_name_items["name"]},  extra = {extra}')
    
    f_name_items["date"], extra = v2.dateFnc(extra)############
    print(f'date = {f_name_items["date"]}')

    isEvent, docuEventExtra = v2.eventFnc(extra) ############### docu, event
    if isEvent == 1 : # 정상
        f_name_items["docu"] = docuEventExtra[0] #사건번호>>docu 할당 #############
        f_name_items["event"] = docuEventExtra[1]
        extra = docuEventExtra[2]
        print(f'isEvent = {isEvent},  docu = {f_name_items["docu"]},  event = {f_name_items["event"]},  extra = {extra}')
    else : # 사건번호 없거나 경정/항고 중 setDocu할 거
        extra = docuEventExtra[2]
        if isEvent == 2 :
            f_name_items["event"] = docuEventExtra[1]
        print(f'isEvent = {isEvent},  docu = {f_name_items["docu"]},  event = {f_name_items["event"]},  extra = {extra}')

        isDocu, docuNameExtra = v2.setDocu(extra) ##############################
        if isDocu :
            f_name_items["docu"] = docuNameExtra[0]#비사건번호>>docu 할당 #############
            extra = docuNameExtra[1]
            print(f"isDocu : {isDocu},  docu : {f_name_items['docu']},  extra : {extra}")
        else : # nodocu
            print("setDocu 결과 nodocu")
    
    extra = v2.rmNeedless(extra)   
    extra = v2.rm_s(extra)
    f_name_items["extra"] = extra

    new_f = "_".join(filter(lambda x :bool(x), f_name_items.values())) + ext
    print(f, ">>>", new_f)
    

-----------------------------------------
name = (주)주원,  extra =  대출서류
date = 
isEvent = 0,  docu = ,  event = ,  extra =  대출서류
setDocu 결과 nodocu
20429252 (주)주원 대출서류.pdf >>> 20429252_(주)주원_대출서류.pdf


In [9]:
# depth test
files = ["20407898 박한수 양도통지서.pdf"]
for f in files:
    new_f = f 
    depth1, depth2, depth3 = v2.setDepth(new_f)########### depth ###########

    print(depth1, depth2, depth3)

wrongdocu  


In [10]:
stem = "망 강충석의한정승인 상속인 강석원_강제집행_2022즈기10012"
name = "강충석"
extra = re.sub("망?\s?"+name+"\s?(의(?=\s))?", "", stem) # 의를 지우되 보증인이 의로 시작하는 경우 있어 (?!\s)추가
print(extra)
# fs = ["20437691 이상래(75) 등기부등본.pdf", "20437691 이상래(75) 법인등기부.pdf", "20437691 이상래(75) 배당표 등본.pdf", "20437691 이상래(75) 결정 등본.pdf"]
# for f in fs : 
#     print(p.sub("aa", f))


의한정승인 상속인 강석원_강제집행_2022즈기10012


In [11]:
def eventFnc(noKeyStem:str)->list:
    """
    키와 확장자 제거한 파일명을 받아 return [0|1|2, old|new stem] \n
    0 : 사건번호 없음 또는 기타에 해당, ["", "", extra] 반환 -> setDocu( ) 호출해 \n
    1 : 사건번호 있고, [docu, event, extra]을 두번째 요소로 반환 \n
    2 : 경정사건번호인데 정확한 문서구분없음 ["", event, extra] 반환 -> setDocu() \n
    """
    new_stem = noKeyStem
    event, eSign, docu, extra = "", "", "", ""
    
    # 컴파일 # 연도의 경우 2030이하면 괜찮음
    p_event1=re.compile(r"((?<=\D)|^)(19\d\d|20[012]\d)\s?(준?재?[가나느차카타즈본징하개회라][가-힣]?)\s?([0-9]+)")
    p_event2=re.compile(r"((?<=\D)|^)([012]\d)\s?(준?재?[가나느차카타즈본징하개회라][가-힣]?)\s?([0-9]+)")
    p_court = re.compile("[가-힣]*(법원|지원|지법|서울|대전|대구|부산|광주|수원|\
        |의정부|파주|포천|동두천|가평|연천|철원|인천|김포|강화|용인|오산|광명|\
        |안성|양평|이천|춘천|홍천|양구|삼척|동해|정선|평창|태백|횡성|인제|화천|\
        |고성|양양|금산|세종|보령|서천|예산|아산|태안|당진|부여|청양|진천|보은|\
        |단양|음성|옥천|괴산|경산|칠곡|청도|영천|성주|고령|영주|봉화|구미|문경|\
        |예천|청송|군위|울진|영양|양산|울산|창원|마산|김해|함안|의령|사천|남해|\
        |하동|거제|창녕|합천|함양|산청|담양|함평|강진|구례|영광|나주|장성|화순|\
        |곡성|광양|고흥|여수|보성|무안|영암|완도|진도|전주|군산|정읍|남원|진안|\
        |김제|무주|임실|익산|부안|고창|장수|순창|제주|서귀포)[가-힣]*")
    # 사해행위는 판결, 품의서, 예고서 등 다방면에 걸쳐있어 키워드로 부적합
    dict = { #사건구분 검색어 / 삭제할 문서구분 키워드
        "집행권원" : [re.compile("가[합단소]|나|다|머|차"), comp["집행권원"]],
        "강제집행" : [re.compile("카(?!경|기|확)|타|즈|본|징|가"), comp["강제집행"]], #카경: 경정, 카기: 기타민사신청, 본(접수증)은 강제집행맞다.
        "개인회생" : [re.compile("개|회"), comp["개인회생"]], # 라 : 민사항고사건
        "파산" : [re.compile("하"), comp["파산"]], 
        "경정" : [re.compile("카경|카기전"), re.compile('집행권원|강제집행|개인회생|파산')],
        "항고" : [re.compile("라"), re.compile('집행권원|강제집행|개인회생|파산')],
        "기타" : [re.compile("느|카(기(?!전)|담|확)"), re.compile("기타")] #느(단) : 한정상속, 상속포기, 기:의사표시공시송달, 담:담보취소, 확:소송비용확정
        }

    # 관할법원 제거
    new_stem = p_court.sub("", new_stem)
    print(new_stem)
    # event, name, extra, eSing 할당
    # 사건번호 없는 경우
    if (p_event1.search(new_stem)==None) and (p_event2.search(new_stem)==None) :
        print("없어")
        return [0, ["", "", noKeyStem]]
    elif comp["기타1"].search(new_stem) :
        print("기타")
        return [0, ["", "", noKeyStem]]
    #사건번호 있는 경우
    elif p_event1.search(new_stem):
        m = p_event1.search(new_stem)
        event = m[2] + m[3] + m[4]
        eSign = m[3]
        extra = new_stem[:m.start()] + new_stem[m.end():]
    else :
        p_event2.search(new_stem)
        m = p_event2.search(new_stem)
        event = "20" + m[2] + m[3] + m[4]
        eSign = m[3]
        extra = new_stem[:m.start()] + new_stem[m.end():]

    return [1, [event, eSign, extra]]

stem = "망 강충석의한정승인 상속인 강석원_강제집행_2022즈기10012"
eventFnc(stem)

망 강충석의한정승인 상속인 강석원_강제집행_2022즈기10012
기타


[0, ['', '', '망 강충석의한정승인 상속인 강석원_강제집행_2022즈기10012']]